Estudo do livro Python Machine Learning de Sebastian Raschka

# Perceptron Algorithm

Algumas fórmulas importantes.

net input: $ z = w_1  x_1 + ... + w_m  x_m $

que pode ser reescrita como $ z = w_0 x_0 + w_1 x_1 + ... + w_m x_m = w^T x $

activation function: $ \phi(z) $

In [11]:
import numpy as np

In [3]:
class Perceptron(object):
    """Perceptron Classifier.
    
    Parameters
    ----------
    eta: float
        Learning rate(between 0.0 and 1.0).
    n_iter: int
        Passes over the training dataset.
    
    Attributes
    ----------
    w_ : 1d-array
        Weights after fitting.
    errors_ : list
        Number of misclassifications in every epoch.
        
    """
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        """Fit training data.
        
        Parameters
        ----------
        X: {array-like}, shape = [n_samples, n_features]
            Training vectors, where 
            n_samples is the number of samples and 
            n_features is the number of features.
            
        y: array-like, shape = [n_samples]
            Target values.
        
        Returns
        -------
        self: object
        
        """
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X,y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi #update the delta wm com n(yi - outputi) * xi
                self.w_[0] += update # update the delta w0 com n(yi - outputi)
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        """Calculate net input"""
        
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)

Explicando por parte. 

- método **predict**. 

Chama o método `net_input` e passa o array. Se o valor retornado é superior ou igual a 0.0 (o **limite** ou em inglês **threshold**) a classe é classificada como **1**, senão ela é classificada como **-1**.

Podendo ser escrito matemáticamente como:

$$   
\phi(z) =
    \begin{cases}
      +1&,  \text{se z >= 0.0} \\
     -1&, \text{caso contrário}
    \end{cases}
$$

- método **net_input**

Faz a multiplicação de matriz do vetor de treinamento **X** com os pesos estipulados. Como o vetor de treinamento não tem o equivalente à $ x_0 w_0 $ e considerando que $ x_0 = 1 $ então só é contado o peso $ w_0 $. 

método **fit**

Obs.: Dúvidas sobre como escrever alguns desses códigos em latex segue uma referência que achei a melhor até agora http://meta.math.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference

Letras gregas 
http://web.ift.uib.no/Teori/KURS/WRK/TeX/sym1.html